# PHYS2015A/PHYS3009A: MeV/GeV Astronomy Project

Name: 

Student No.:

total marks: 

In this project you will do yourself the data analysis of the object RX J1713.7-3946. 

You will need to answer questions and write code.

Please put your code in the cells starting with 

```
# your code here
```

You will also need to answer some questions for marks. Edit the text within the cells, either providing descriptions of labeled parameters, or typing your answer where it says "Answer here".

Do not change the code in any other cells. If you want to add additional ouput, or test some things you can create new cells for that.

# Likelihood Analysis with fermipy

The python likelihood tools are a very powerful set of analysis tools that expand upon the command line tools provided with the Fermi Science Tools package. Not only can you perform all of the same likelihood analysis with the python tools that you can with the standard command line tools but you can directly access all of the model parameters. You can more easily script a standard analysis. There are also a few things built into the python tools that are not available from the command line like the calculation of upper limits.

There are many user contributed packages built upon the python backbone of the Science Tools and this thread will highlight the use of the [fermipy](http://fermipy.readthedocs.org) package.

This sample analysis of RX J1713.7-3946 is based on the PG 1553+113 analysis performed by the LAT team and described in [Abdo, A. A. et al. 2010, ApJ, 708, 1310](http://adsabs.harvard.edu/abs/2010ApJ...708.1310A) and closely follows the [Likelihood Analysis with Python](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/python_tutorial.html) thread.  This tutorial assumes you have the most recent ScienceTools installed and [fermipy](http://fermipy.readthedocs.org) installed on top of it.  For instructions on installing fermipy and the Fermi ScienceTools you should consult the [fermipy Installation Instructions](http://fermipy.readthedocs.org/en/latest/install.html).  We will also make significant use of python, so you might want to familiarize yourself with python including matplotlib and other libraries (there's a beginner's guide at http://wiki.python.org/moin/BeginnersGuide). This tutorial also assumes that you've gone through the non-python based [Unbinned Likelihood Tutorial](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/likelihood_tutorial.html).  

## Get the Data

For this thread the original data were extracted from the [LAT data server](http://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections (these selections are similar to those in the 4FGL-DR4 paper):

* Search Center (RA,Dec) = (258.112,-39.687)
* Radius = 15 degrees
* Start Time (MET) = 239557417 seconds (2008-08-04T15:43:36)
* Stop Time (MET) = 681169985 seconds (2022-08-02T21:53:00)
* Minimum Energy = 50 MeV
* Maximum Energy = 1000000 MeV

Let's retrieve the data. In this example the output of the analysis will go into a subdirectory called *data*.

In [ ]:
import os
import subprocess
import gdown

if os.path.isdir('data'):
    os.environ['DATA_DIR'] = 'data'
else:
    os.mkdir('data')
    os.environ['DATA_DIR'] = 'data'

drive_file_IDs = [
["1AVK4mR-KfC0R4Be8bQ8paf1LPEi4fa9P","data/L230831165314B24E85D632_PH00.fits"],
["1AylrtavbJMaGADh-bEJ6y9Z35d-q-hFk","data/L230831165314B24E85D632_PH01.fits"],
["1B6Ogp9qA9Yb1TZ-AnqSIam3CEsccSFOC","data/L230831165314B24E85D632_PH02.fits"],
["1B-1OGD9GtMK-_98AmNz-To7njJqzlUN8","data/L230831165314B24E85D632_PH03.fits"],
["1B8LKOot0ZKER2J5FaEFAK6fGM1k2FZU8","data/L230831165314B24E85D632_PH04.fits"],
["1B0iCHiwtRvjdd9W-9HWB0B0Y6Okz5SC3","data/L230831165314B24E85D632_PH05.fits"],
["1m_XWgvAZdqxTjBY8Sa6uiLlJ3Fl4md4T","data/L230831165314B24E85D632_SC00.fits.gz"],
["1mc5k5ZG_f0_QBoc9j7Xs6Z8pq1UqU_8b","data/ltcube_00.fits"]
]
for file in drive_file_IDs :
    if not os.path.isfile(file[1]):
        gdown.download(id=file[0],output=file[1],quiet=False)
subprocess.run(["gunzip","data/L230831165314B24E85D632_SC00.fits.gz"]) 


Now let's retrieve the diffuse emission models. These come directly from the *fermipy-extras* package. We will put them in the *diffuse* subdirectory.

In [ ]:
import os

if os.path.isdir('data'):
    os.environ['DATADIR'] = 'data'

if os.path.isfile('diffuse.tar.gz'):
    !tar xzf diffuse.tar.gz
else:
    !curl -OL https://raw.githubusercontent.com/fermiPy/fermipy-extras/master/data/diffuse.tar.gz
    !tar xzf diffuse.tar.gz

### Make a file list

You'll then need to make a file list with the names of your input event files. You can either just make one with a text editor or do the following from the command line.

In [ ]:
!ls -1 data/*PH*.fits > data/RXJ1713.7-3946.lst

## Make a config file

fermipy bases its analysis on a configuration file (in [yaml](http://yaml.org) format).  We're just going to use a really simple config file for a standard analysis.  There are many many more options which you can use or you can modify these options after the fact within the analysis chain.


Make a config file named 'config.yaml' like the following.  For more details on the config file see [config.html](http://fermipy.readthedocs.org/en/latest/config.html).  You will probably need to customize this a bit since your files might not be in the same place or named the same.  The galactic and isotropic diffuse will need to be located on your system (they are included in the science tools or can be downloaded from the FSSC).  In the following example we set the path to these files with the environment variable FERMI_DIFFUSE_DIR.  If FERMI_DIFFUSE_DIR is not defined fermipy will look for the location of these files within the FSSC STs distribution. 

```
data:
  evfile : RXJ1713.7-3946.lst
  scfile : data/L211025154921445A2D9929_SC00.fits

binning:
  roiwidth   : 10.0
  binsz      : 0.1
  binsperdec : 8

selection :
  emin : 100
  emax : 300000
  tmin : 239557417
  tmax : 681169985
  zmax    : 105
  evclass : 128
  evtype  : 3
  target : 'RXJ1713.7-3946'

gtlike:
  edisp : True
  irfs : 'P8R2_SOURCE_V6'
  edisp_disable : ['isodiff','galdiff']

model:
  src_roiwidth : 15.0
  galdiff  : 'diffuse/gll_iem_v07.fits'
  isodiff  : 'diffuse/iso_P8R2_SOURCE_V6_v06.txt'
  catalogs : ['4FGL']
```

## Start the analysis

Next, you create an analysis script and run the setup steps which include running the selections and generating exposure maps etc.  This will take a bit.

This is where the magic happens.  fermipy will load the point source model, create your xml file for you, decide on all the appropriate cuts and binnings and just go.  All of this is configurable from python or from the config file.  And, if you need to rerun things, it's smart enough to not overwrite files if it doesn't need to.

### Load up some useful modules

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

### Import the GTAnalysis module from fermipy

You start by importing the module and then creating an instance of the analysis object from our config file.  When instantiating the analysis object we can override any options defined in the configuration file by passing keyword arguments to the object constructor.  Here we explicitly set the verbosity parameter to 3 (INFO) which supresses DEBUG output.  When we create the object, it spits out a bunch of information about all of the parameters that were used.  You can see there are many more options than the ones we chose.

In [ ]:
from fermipy.gtanalysis import GTAnalysis
gta = GTAnalysis('data/config.yaml',logging={'verbosity': 3})
matplotlib.interactive(True)

### The setup routine

This gets everything ready for the likelihood analysis including instantiating the pylikelihood object.  Note that fermipy will skip generating any ancillary files that already exist in the working directory.  In the sample tarball these files have already been produced in order to speed up this stage of the analysis.  If you want to see the behavior of fermipy when running from an empty working directory you can delete one or more of these files before running *setup*.

In [ ]:
# your code here

Output produced: **[1 mark]**

Before proceeding with the analysis we'll have a quick look at the files that are produced by the setup function.

In [ ]:
ls data/*.fits

For each of the following files, write a brief explanation of the contents of each file and its role in the analysis: **[7 marks]**

* **ft1_00.fits**: 
* **bexpmap_00.fits**: 
* **bexpmap_roi_00.fits**: 
* **ccube_00.fits**: 
* **ltcube_00.fits**: 
* **srcmap_00.fits**: 

To see example of one of these files we can open and plot the counts cube file.  This is a 3D cube that contains the distribution of events as a function of energy and two spatial coordinates.  In the example below we sum over the energy dimension of the cube to make a 2-D sky image.

In [ ]:
import astropy.io.fits as pyfits

h = pyfits.open('data/ccube.fits')
h.info()
counts = h[0].data
counts.shape
plt.figure()
plt.imshow(np.sum(counts,axis=0),interpolation='nearest',origin='lower')

We can now inspect the state of the ROI prior with the print_roi() method.

In [ ]:
# your code here

Output produced: **[1 mark]**

How many catalog point sources are there within the ROI? **[1 mark]**

Answer here:


Additional details about an individual source can be retrieved by printing the corresponding source object.  Here we use the bracket operator to return the properties of RXJ1713.7-3946. The catalog name of PKS 2155-403 is '4FGL J1713.5-3945e'.

In [ ]:
source_name = '4FGL J1713.5-3945e'
# your code here

Output produced: **[1 mark]**

Notice that the SpatialModel is SpatialMap rather than PointSource. An intensity map taken from another instrument (in this case HESS) is used as input to the model rather than assuming that all of the photons come from only one direction. The shape and relative intensities are fixed; only the normalization is left as a free parameter. Let's take a look at the template used for this source.

In [ ]:
h2 = pyfits.open('data/RXJ1713_2016_250GeV.fits')
h2.info()
template = h2[0].data
template.shape
plt.figure()
plt.imshow(template,interpolation='nearest',origin='lower')

Does the spatial map change the number of free parameters in the model compared to a point source? Why or why not? **[1 mark]**

Answer here:

## Do the likelihood fitting

Now that all of the ancillary files have been generated, we can move on to the actual fitting.  The first thing you should do is free some of the sources since all of the sources are initially fixed.  We'll just free those sources in the center region.

In [ ]:
# Free Normalization of all Sources within 3 deg of ROI center
# your code here

# Free all parameters of isotropic and galactic diffuse components
# your code here (two lines)

Output produced: **[3 marks]**

What is the purpose of freeing the normalization of sources within 3 degrees? How are the normalizations affected when fitting the null hypothesis? **[2 mark]**

Answer here:

In this simple anlaysis we are leaving the spectral shapes of sources fixed but we're going to free the spectral shape of the source we care about.  

In [ ]:
# your code here

Output produced: **[1 mark]**

Now, actually do the fit.  The software does its best to get the fit to converge by running the fit several times.

In [ ]:
# your code here

Output produced: **[1 mark]**

Let's look at the fit results and the quality of the fit.

In [ ]:
print('Fit Quality: ',fit_results['fit_quality'],', Fit status: ',fit_results['fit_status'])

What is the significance of these two numbers? What do we expect them to be? **[4 marks]**

Answer here:


Let's see if we can improve the quality of the fit by running it again, with starting values obtained from the previous fit.

In [ ]:
# your code here

Output produced: **[1 mark]**

The dictionary returned by the fit method returns a variety of diagnostic information about the fit including the fit quality, the relative improvement in the likelihood, and the correlations among the fit parameters.  We can inspect the results of the fit by printing the source object for RXJ1713.7-3946.

In [ ]:
print('Fit Quality: ',fit_results['fit_quality'],', Fit status: ',fit_results['fit_status'])
print(gta.roi[source_name])

Explain the meaning of the following parameters: **[7 marks]**


* **TS**             : 
* **Npred**          : 
* **Flux**           : 
* **EnergyFlux**     : 
* **b'norm'**        :  
* **b'alpha'**       :       
* **b'beta'**        :    
* **b'Eb'**          :



You can then save the state of the roi to an output file for reference later.  The write_roi function does this.  The first argument is a string that will be prepended to the names of the output files generated by this method.

In [ ]:
## your code here

Output produced: **[1 mark]**

There are a lot of diagnostic plots also saved at the same time.  

In [ ]:
ls -l data/*.png

In [ ]:
from IPython.display import Image, display
from glob import glob

In [ ]:
pngs = glob('data/*.png')

In [ ]:
for png in pngs:
    my_image = Image(png)
    display(my_image)

Provide a short description of each of the five figures: **[5 marks]**

Answer here:

What is the dominant component of the spectrum **[1 mark]**

Answer here:

### Reading in the results

Since the results are saved, you can load them back up at any point (you can also get to these within python).  Here we retrieve the analysis results from the output numpy file. 

In [ ]:
c = np.load('data/fit0.npy', allow_pickle=True).flat[0]

The `sources` dictionary has an entry for each source in the model:

In [ ]:
sorted(c['sources'].keys())

Let's take a look at the flux, spectral parameters, and TS.

In [ ]:
c['sources'][source_name]['flux']

In [ ]:
print(c['sources'][source_name]['param_names'][:4])
print(c['sources'][source_name]['param_values'][:4])

In [ ]:
c['sources'][source_name]['ts']

The SED is in there as well.  We can plot it.

What does SED stand for? **[1 mark]**

Answer here:

In [ ]:
E = np.array(c['sources'][source_name]['model_flux']['energies'])
dnde = np.array(c['sources'][source_name]['model_flux']['dnde'])
dnde_hi = np.array(c['sources'][source_name]['model_flux']['dnde_hi'])
dnde_lo = np.array(c['sources'][source_name]['model_flux']['dnde_lo'])

In [ ]:
plt.loglog(E, (E**2)*dnde, 'k--')
plt.loglog(E, (E**2)*dnde_hi, 'k')
plt.loglog(E, (E**2)*dnde_lo, 'k')
plt.xlabel('E [MeV]')
plt.ylabel(r'E$^2$ dN/dE [MeV cm$^{-2}$ s$^{-1}$]')
plt.show()

If you want SED points, there's a function for that.  There are lots of options for this which you can set in the config file or from keyword arguments of the function itself.

In [ ]:
# your code here

Output produced: **[1 marks]**

You can save the state to the yaml file or you can just access it directly.  This is also the way to get at the dictionary for any individual source.

In [ ]:
src = gta.roi[source_name]

In [ ]:
plt.loglog(E, (E**2)*dnde, 'k--')
plt.loglog(E, (E**2)*dnde_hi, 'k')
plt.loglog(E, (E**2)*dnde_lo, 'k')
plt.errorbar(np.array(sed['e_ctr']),
             sed['e2dnde'], 
             yerr=sed['e2dnde_err'], fmt ='o')
plt.xlabel('E [MeV]')
plt.ylabel(r'E$^{2}$ dN/dE [MeV cm$^{-2}$ s$^{-1}$]')
plt.show()

In [ ]:
from astropy.table import Table
sed_tab = Table.read('data/4fgl_j1713.5-3945e_sed.fits')
sed_tab.write('data/4fgl_j1713.5-3945e_sed.ecsv',exclude_names=['norm_scan','dloglike_scan'],overwrite=True)

### Summary

This is it. You have done an analysis of the data recorded with Fermi-LAT on RXJ1713.7-3946.

### Sanity Check

Before you submit your work you should make a few checks that everything works fine.

1. Save your notebook as a PDF (File->Print...->Print as PDF). This document will help you debugging in the next step.
1. Restart the kernel and rerun the entire notebook (Kernel->Restart Kernel & Run All Cells...). This will delete all variables (but not your code) and rerun the notebook in one go. If this does not go through the end (i.e. you do not see the output of the next cell) then you have to fix it. You will see at which cell the run stopped. A common mistake is using a variable that is defined only at a later stage.
1. You think you fixed everything? Redo step 2 (Kernel->Restart Kernel & Run All Cells...)

In [ ]:
print('a\bYa\boa\bua\b a\baa\bra\bea\b a\bra\bea\baa\bda\bya\b a\bta\boa\b a\bsa\bua\bba\bma\bia\bta\b.a\b')

Do you see the output of the last cell without explicitly running it? Then the notebook goes through with one kernel restart. You can proceed to submission.
You do not see the output? Go back to step 2 above.

The jupyter notebook goes through all cells with one Kernel->Restart Kernel & Run All Cells...    **[1 mark]**

# Submission
You have to download and submit 3 files.
- PDF file. File->Print...->Print as PDF. Save this file on your disk. If this doesn't work you can also try File->Export Notebook As->HTML. This file serves as reference for what you have done.
- Jupyter notebook. File->Download. Save this file on your disk. This file will be tested and you receive your marks for that.
- Spectral data points. Go to the Home Page of your jupyter notebooks. It is in a different tab or window of your browser. Find the file `data/4fgl_j1713.5-3945e_sed.ecsv` and save it to your disk. You will need this file at a later stage.

Please submit all 3 files on Ulwazi.        **[1 mark]**

Congratulations. You have succesfully completed your MeV/GeV Astronomy project.